In [ ]:
#libraries

import pandas as pd
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt
import talib as ta
import seaborn as sns
#from datetime import datetime
import glob
import os
#import subprocess
sns.set_theme()

pd.set_option('display.max_columns', None);pd.set_option('display.max_rows', None)


In [ ]:
def plot_with_mpl(root,tick,day=60,sty='classic',typ='candle'):
    "returns the candle stick of a stock tick for the given root file."

    df=pd.read_csv(root+'/'+tick+'.csv').tail(day)
    df['Date']=df['Date'].apply(lambda x:x.split()[0])

    df['rsi']=ta.RSI(df['Close']);df['sma-5']=ta.SMA(df['Close'],5);df['sma-10']=ta.SMA(df['Close'],10)
    #df.fillna(0,inplace=True)
    apds = [ 
         mpf.make_addplot((df['rsi']),panel=1,color='r',secondary_y=True,type='line',width=0.5),
         #mpf.make_addplot((df['sma-5']),color='g',type='line',width=0.2),
         #mpf.make_addplot((df['sma-10']),color='r',type='line',width=0.2)
       ]
    df.Volume=df.Volume/10**6              #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    fig,axes=mpf.plot(df,type=typ,mav=(5,10,21),volume=True,tight_layout=False,figratio=(80,36),figscale=1,
                    style=sty,returnfig=True,addplot=apds,scale_width_adjustment=dict(volume=0.6,candle=0.8,lines=0.4))#,update_width_config=dict(candle_linewidth=1.75,candle_width=0.1))
    axes[0].legend([" "," ","5-SMA","10-SMA","21-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
    axes[0].set_title(tick)
    mpf.show()

# def make_clickable(url, name):
#     return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,name)

def get_yahoo_link(ticker):
    """
    Get the yahoo link
    """
    #return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,name)
    url="https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch"
    #return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,ticker)
    return url.format(ticker,ticker)

def short_industry(x):
    temp=x.split()
    if len(temp)>2:
        return temp[0]+"-"+temp[1]
    else:
        return x


In [ ]:
def plot_all(j,plot=True):
    #j=0 #nano
    df=pd.read_csv(files[j]).drop(columns=['SMA150','SMA200PAST20','NAME']).sort_values(by='CP').reset_index(drop=True)
    df['YAHOO']=df['TICK'].apply(lambda x:get_yahoo_link(x))#.apply(lambda x: yahoo_clickable(x, 'yahoo'))

    #df_sector=reset_index(df_sector)
    print(df.to_string())
    tick=df['TICK'].to_list()
    #sec=df_sector['SECTOR'].to_list()
    root=roots[j]
    if plot:
        for c,i in enumerate(tick):
        #file=root.format(cat[c],i)
        #print(f"file: {file}")
            print(f"\n\n{df[df['TICK']==i].to_string()}\n")
            plot_with_mpl(root,i,sty='default')

In [ ]:
#create yfinance portfolio sample
def yahoo_portfolio(df,save_name):
    """
    returns the yfinance compatable portfolio template for given df name of save file
    """
    cols=['Symbol','Current Price','Date', 'Time', 'Change', 'Open', 'High', 'Low', 'Volume', 'Trade Date',
          'Purchase Price','Quantity','Commission','High Limit','Low Limit','Comment']

    sym=list(df.TICKER)
    dic_val=[sym]+14*[None] #14 None
    my_dict=dict(zip(cols,dic_val)) #dictionary
    my_df=pd.DataFrame.from_dict(my_dict)
    my_df.to_csv(save_name+".csv")
    print(f"Created the yahoo template: {save_name}.csv from data frame !")

In [ ]:
root='/home/thakur/test/combined/{}'


In [ ]:
all_nasdaq='all_nasdaq.csv'
#all_mark='all_mark.csv'



# ALL NASDAQ

In [ ]:
df_nasdaq=pd.read_csv(root.format(all_nasdaq))
df_nasdaq.head()

# SELECTED

In [ ]:
sel_list=['MSFT','AAPL','GOOGL','META','NVDA','TSLA','AMZN']

In [ ]:
#test=df_mark.TICK.to_list()
df=df_nasdaq[df_nasdaq.TICKER.isin(sel_list)]\
.reset_index(drop=True)
df.index+=1
df

# SORT BY RSI

In [ ]:
by='RSI'
df.sort_values(by=by,ascending=False).reset_index(drop=True)




# SORT BY %CHG

In [ ]:
by='%CHG'
df.sort_values(by=by,ascending=False).reset_index(drop=True)

# SORT BY AVG VOLUME


In [ ]:
by='AVGVOL'
df.sort_values(by=by,ascending=False).reset_index(drop=True)

# BY VOLUME

In [ ]:
by='VOL'
df.sort_values(by=by,ascending=False).reset_index(drop=True)

# BY %ATR


In [ ]:
df['%ATR']=round(df['ATR']*100/df['CP'],2)
by='%ATR'
df=df.sort_values(by=by,ascending=False).reset_index(drop=True)
df

In [ ]:
tickers=df.TICKER.to_list()



r='/home/thakur/test/{}'
for i in tickers:
    root=df.loc[df.TICKER==i,"CATEGORY"].values[0]
    root=r.format(root)
    plot_with_mpl(root,i,sty='default')

In [ ]:
ticker='TSLA'
category='mega'
file=f'/home/thakur/test/{category}/{ticker}.csv'
print(f"file: {file}")
df=pd.read_csv(file)
df['Volume']=round(df['Volume']/1e6,2)
df.tail(20)

In [ ]:
df['C-O']=df['Close']-df['Open'];
df['H-O']=df['High']-df['Open'];
df['O-L']=df['Open']-df['Low'];
df['H-L']=df['High']-df['Low'];

In [ ]:
df.tail(20)

In [ ]:
df['Day']=df['Date'].apply(lambda x:pd.Timestamp(x).day_name())

In [ ]:
df.tail(20)

In [ ]:
# close higher than open
c=sum(df['C-O']>0)
o=sum(df['C-O']<0)

print(f"Close > Open: {c}")
print(f"Close < Open: {o}")

In [ ]:
grouped=df.groupby('Day')
days=list(grouped.groups)
days

In [ ]:
df=grouped.get_group('Monday').tail(20).reset_index(drop=True)
df

In [ ]:
def get_day_info(df):
    sel_col=['Date', 'Volume', 'C-O', 'H-O', 'O-L', 'H-L', 'Day']
    temp_df=df[sel_col]
    temp_df.set_index('Date')
    #day
    day=temp_df.Day.values[0]
    mean_vol=temp_df['Volume'].mean().round(2)
    co_net=temp_df['C-O'].mean().round(2)
    co_abs=temp_df['C-O'].abs().mean().round(2)

    ho_net=temp_df['H-O'].mean().round(2)
    ol_net=temp_df['O-L'].mean().round(2)
    hl_net=temp_df['H-L'].mean().round(2)



    print(f"""
    Day        : {day}
    volume     : {mean_vol} 
    co_net     : {co_net}
    co_abs     : {co_abs}
    ho_net     : {ho_net}
    ol_net     : {ol_net}
    hl_net     : {hl_net}
    """)
    return [day,mean_vol,co_net,co_abs,ho_net,ol_net,hl_net]

In [ ]:
get_day_info(df)

In [ ]:
days=['Monday',  'Tuesday', 'Wednesday','Thursday','Friday']
rows=20
all_list=[]
for i in days:
    df=grouped.get_group(i).tail(rows).reset_index(drop=True)
    print(df.to_string())
    info_list=get_day_info(df)
    all_list.append(info_list)

# SINGLE DATA FRAME

In [ ]:
columns=['day','vol','co_net','co_abs','ho_net','ol_net','hl_net']
df=pd.DataFrame(all_list,columns=columns)

In [ ]:
df.set_index('day',inplace=True)

In [ ]:
df

In [ ]:
ax=df.plot.bar(subplots=False,figsize=(20,10),logy=True)
for container in ax.containers:
    ax.bar_label(container)
plt.tight_layout()
plt.show()